In [1]:
import timbermafia
import logging
import sys
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
def test_log_with_a_very_long_name():
    log.debug('test from func')
class MyClass(timbermafia.Logged):
    def status(self):
        self.log.info(f'logging from {self.__class__.__name__} in the function status')

In [2]:
timbermafia.configure(bold=False, style='simple_mono', palette='dawn')
timbermafia.add_handler(stream=sys.stdout, filename='/tmp/timbermafia_test.log', clear=True)

log = logging.getLogger('testing')
log.header('Demo of timbermafia logging')
log.info('INFO messages look like this')
log.info('urls look like this: www.github.com or https://ipleak.net/')
log.info('local files look like this: /tmp/timbermafia_test.log '
         '- this output is being written there too!')
log.info(lorem)

m = MyClass()
m.log.info('messages from MyClass with a timbermafia mixin logger look like this')
m.status()
log.debug('DEBUG messages look like this')
log.warning('WARNING messages look like this')
log.error('ERROR messages look like this')
log.fatal('FATAL/CRITICAL error messages look like this.')
test_log_with_a_very_long_name()


{'level': 10, 'style': 'simple_mono', 'palette': 'dawn', 'monochrome': True, 'bold': False, 'enclose': False, 'enclosers': [], 'show_separator': True, 'divide_lines': False, 'line_separator': '-', 'separator': '>', 'separator_style': 'smart', 'sparse_separators': True, 'clean_names': True, 'justify': 'left', 'justify_left': ['message'], 'justify_right': [], 'justify_center': [], 'truncate': [], 'columns': 141, 'name_padding': 8, 'funcName_padding': 8, 'module_padding': 25, 'pathname_padding': 40, 'lineNo_padding': 4, 'thread_padding': 15, 'threadName_padding': 10, 'format': '{levelname} > {name}.{funcName} > {message}', 'time_format': '%H:%M:%S', 'format_style': '{', 'sparse_separator': True}
---------------------------------------------------------------------------------------------------------------------------------------------
>                                                        Demo of timbermafia logging                                                        >
--------------

{'max_lines': 1, 0: {'line_array': ['DEBUG   '], 'padding': 8, 'section': '{levelname}'}, 1: {'line_array': ['testing         '], 'padding': 16, 'section': '{name}.{funcName}'}, 2: {'line_array': ['DEBUG messages look like this                                                                                  '], 'padding': 111, 'section': '{message}'}}
{'max_lines': 1, 0: {'line_array': ['WARNING '], 'padding': 8, 'section': '{levelname}'}, 1: {'line_array': ['testing         '], 'padding': 16, 'section': '{name}.{funcName}'}, 2: {'line_array': ['WARNING messages look like this                                                                                '], 'padding': 111, 'section': '{message}'}}
WARNING  > testing          > WARNING messages look like this                                                                                
{'max_lines': 1, 0: {'line_array': ['WARNING '], 'padding': 8, 'section': '{levelname}'}, 1: {'line_array': ['testing         '], 'padding': 16, 'sect

In [3]:
# f = log.parent.handlers[0].formatter
# f._fmt
# f.separator

In [4]:
# l = logging.LogRecord('name', logging.DEBUG, '', 21, 'my title', [], False, func='my_func')
# l.getMessage()
# l.created